In [2]:
import pandas as pd
df = pd.read_csv("all_missions_standardized.csv")

tf = df["label"]


df = df.fillna(0)

df.drop(["star_id", "mission","label"], axis=1, inplace=True)

tf.head()

0         CONFIRMED
1         CANDIDATE
2    FALSE_POSITIVE
3         CONFIRMED
4         CONFIRMED
Name: label, dtype: object

In [ ]:
#Scaling the training data to a range of 0 to 1
from sklearn.preprocessing import MinMaxScaler,LabelEncoder
scaler = MinMaxScaler(feature_range=(0, 1))
scaler.fit_transform(df)

le = LabelEncoder()
y = le.fit_transform(tf)


#Splitting the data into a train section and test section
from sklearn.model_selection import train_test_split
Train_X, Test_X, Train_Y, Test_Y = train_test_split(df, y)

from catboost import CatBoostClassifier

model = CatBoostClassifier(
    loss_function="MultiClass",
    iterations=1500,
    depth=8,
    learning_rate=0.05,
    random_seed=42,
    verbose=False
)


""""
from lightgbm import LGBMClassifier

model = LGBMClassifier(
    objective="multiclass",
    num_class=3,
    n_estimators=800,
    learning_rate=0.05,
    subsample=0.9,
    colsample_bytree=0.9,
    random_state=42,
    n_jobs=-1
)



from xgboost import XGBClassifier

model = XGBClassifier(
    objective="multi:softprob",
    num_class=3,
    eval_metric="mlogloss",
    tree_method="hist",     
    n_estimators=700,
    learning_rate=0.05,
    max_depth=6,
    subsample=0.9,
    colsample_bytree=0.9,
    random_state=42,
    n_jobs=-1
)
"""
model.fit(Train_X,Train_Y)
Pred_Y = model.predict(Test_X)



from sklearn.metrics import accuracy_score
print(accuracy_score(Pred_Y,Test_Y))

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000770 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2295
[LightGBM] [Info] Number of data points in the train set: 12886, number of used features: 9
[LightGBM] [Info] Start training from score -0.829098
[LightGBM] [Info] Start training from score -1.608352
[LightGBM] [Info] Start training from score -1.012416
0.715316573556797


In [6]:
import pickle

with open('LGBM.pkl','wb') as file: 
    pickle.dump(model, file)

with open('LGBM_scaler.pkl','wb') as file: 
    pickle.dump(scaler, file)
